<font size=8>&nbsp;<font color=#F7C5A1>**XDCClab =**</font> <font color=#F9AB00>**Colab**<font color=#C9E9FF> **+**<font color=#908672> **xdcc-dl**<font color=#C9E9FF> **+** <font color=#017547>**xdccget**

###### In case you stumbled upon this colab notebook somewhere on the internet, here's the **[repo](https://github.com/hyPnOtICDo0g/Otakulab)**.

## Configuring the IRC clients
Upload your rclone config file and run this cell. ***IRC gang, where you at?***

In [ ]:
#@markdown #### Fetch Dependencies
%%shell

if ! [ -e /content/rclone.conf ]
then
  echo -e "\e[31mRclone config not found."
  # using exit code 0 instead of 1 as colab tends to throw a ton of errors
  exit 0
fi

if ! [ -e /usr/bin/xdccget ]
then
  git clone -q https://github.com/RhythmLunatic/xdccget.git
  cd xdccget && make -s && mv xdccget /usr/bin
fi

rm -rf "/content/sample_data" 
mkdir -p "/content/Downloads"
pip install xdcc-dl -q
curl https://rclone.org/install.sh | sudo bash
echo -e "\033[96mDependencies Fetched."

## Using a required client to download
***Feeling nostalgic yet?***

In [ ]:
#@markdown #### • xdcc-dl will be set as default as it's stable & works great for batch downloads.
#@markdown #### • xdccget is more verbose, i.e. it includes a speed indicator, queue position, etc but no batch support.

CLIENT = 'xdcc-dl' #@param ["xdcc-dl", "xdccget"]
CHANNEL = '#nibl' #@param ["#nibl", "#MK", "#XDCCLeech"] {allow-input: true}
SERVER = 'irc.rizon.net' #@param ["irc.rizon.net", "irc.xertion.org"] {allow-input: true}
COMMAND = '' #@param {type:"string"}
DIR = '/content/Downloads'

if COMMAND != '':
# xdcc-dl tends to give an error when a download a forcefully interrupted, this can be safely ignored
  if CLIENT == 'xdcc-dl':
    cmd = COMMAND.replace('batch ', 'send #')
    !cd "$DIR" && xdcc-dl -v "$cmd" -s "$SERVER" --fallback-channel "$CHANNEL"
  elif CLIENT == 'xdccget':
    cmd = COMMAND.split()[1] + ' xdcc send ' + COMMAND.split()[4]
    !xdccget -v -i "$SERVER" "$CHANNEL" "$cmd" --dont-confirm-offsets -d "$DIR"
else:
  print('\033[91mEmpty \'COMMAND\' Field.')

## The Rclone Wars
***Pew Pew*** those downloaded files to your remote.

In [ ]:
#@markdown #### • Enter the source & destination paths & run this cell.
#@markdown #### • You can obtain the **source path** by using the file manager to your left.
#@markdown #### • **Destination path** is the remote you want to transfer to.

SOURCE = "/content/Downloads" #@param {type:"string"}
DESTINATION = "drive:" #@param {type:"string"}
ARGS = "--transfers=10 --fast-list --stats-one-line -v" #@param {type:"string"}

!rclone copy "$SOURCE" "$DESTINATION" $ARGS --config rclone.conf